In [4]:
# Tensorflow / Keras
import tensorflow as tf
from tensorflow import keras 
from keras.models import Sequential # creating a linear stack of NN
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout 

# data manipulation
import pandas as pd
import numpy as np

# sklearn 
import sklearn 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OrdinalEncoder

# visualisation 
import cv2 
import matplotlib
import matplotlib.pyplot as plt

